#### LeetCode - SQL - #1393

Refer [1393. Capital Gain/Loss](https://leetcode.com/problems/capital-gainloss/description/)

Write a solution to report the  **Capital gain/loss**  for each stock.

The  **Capital gain/loss**  of a stock is the total gain or loss after buying and selling the stock one or many times.

Return the result table in  **any order**.

In [0]:
data_stock = [['Leetcode', 'Buy', 1, 1000], ['Corona Masks', 'Buy', 2, 10], ['Leetcode', 'Sell', 5, 9000], ['Handbags', 'Buy', 17, 30000], ['Corona Masks', 'Sell', 3, 1010], ['Corona Masks', 'Buy', 4, 1000], ['Corona Masks', 'Sell', 5, 500], ['Corona Masks', 'Buy', 6, 1000], ['Handbags', 'Sell', 29, 7000], ['Corona Masks', 'Sell', 10, 10000]]
columns_stock = ['stock_name', 'operation', 'operation_day', 'price']
schema_stock = {'stock_name':'object', 'operation':'object', 'operation_day':'Int64', 'price':'Int64'}

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('Leetcode SQL').getOrCreate()

In [0]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_stock = pandas.DataFrame(data=data_stock, columns=columns_stock).astype(schema_stock)

df_stock = spark_context.createDataFrame(pandas_df_stock)
df_stock.printSchema()
df_stock.show(5)

In [0]:
# Create Spark Dataframe using PySpark functions - 1
schema_stock = 'stock_name string, operation string, operation_day integer, price integer'

df_stock = spark_context.createDataFrame(data=data_stock, schema=schema_stock)
df_stock.printSchema()
df_stock.show(5)

In [0]:
# Create Spark Dataframe using PySpark functions - 2
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema_stock = StructType([
    StructField('stock_name', StringType(), False),
    StructField('operation', StringType(), False),
    StructField('operation_day', IntegerType(), False),
    StructField('price', IntegerType(), False)
])

df_stock = spark_context.createDataFrame(data=data_stock, schema=schema_stock)
df_stock.printSchema()
df_stock.show(5)

In [0]:
from pyspark.sql import functions as F

df_stock.groupBy(df_stock.stock_name).agg(
    F.sum(
        F.when(df_stock.operation.eqNullSafe('Buy'), -(df_stock.price)).otherwise(df_stock.price)
    ).alias('capital_gain_loss')
).show()

+------------+-----------------+
|  stock_name|capital_gain_loss|
+------------+-----------------+
|    Leetcode|             8000|
|Corona Masks|             9500|
|    Handbags|           -23000|
+------------+-----------------+

